In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
Pkg.add("Distributions")
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
# using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo
using Distributions

  Activating project at `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`


In [2]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [3]:
"""
x = fmincon(cost,equality_constraint,inequality_constraint,x_l,x_u,c_l,c_u,x0,params,diff_type)

This function uses IPOPT to minimize an objective function 

`cost(params, x)` 

With the following three constraints: 

`equality_constraint(params, x) = 0`
`c_l <= inequality_constraint(params, x) <= c_u` 
`x_l <= x <= x_u` 

Note that the constraint functions should return vectors. 

Problem specific parameters should be loaded into params::NamedTuple (things like 
cost weights, dynamics parameters, etc.). 

args:
    cost::Function                    - objective function to be minimzed (returns scalar)
    equality_constraint::Function     - c_eq(params, x) == 0 
    inequality_constraint::Function   - c_l <= c_ineq(params, x) <= c_u 
    x_l::Vector                       - x_l <= x <= x_u 
    x_u::Vector                       - x_l <= x <= x_u 
    c_l::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    c_u::Vector                       - c_l <= c_ineq(params, x) <= x_u 
    x0::Vector                        - initial guess 
    params::NamedTuple                - problem parameters for use in costs/constraints 
    diff_type::Symbol                 - :auto for ForwardDiff, :finite for FiniteDiff 
    verbose::Bool                     - true for IPOPT output, false for nothing 

optional args:
    tol                               - optimality tolerance 
    c_tol                             - constraint violation tolerance 
    max_iters                         - max iterations 
    verbose                           - verbosity of IPOPT 

outputs:
    x::Vector                         - solution 

You should try and use :auto for your `diff_type` first, and only use :finite if you 
absolutely cannot get ForwardDiff to work. 

This function will run a few basic checks before sending the problem off to IPOPT to 
solve. The outputs of these checks will be reported as the following:

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :auto (ForwardDiff.jl)----
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

If you're getting stuck during the testing of one of the derivatives, try switching 
to FiniteDiff.jl by setting diff_type = :finite. 
""";

In [4]:
# import piano model
model = load_model("/home/aipex6/Conan/piano project/mujoco_menagerie/shadow_hand/scene_right_modified.xml")
data = init_data(model)

MuJoCo Data object

In [5]:
println("number of geoms: ",model.ngeom)
finger_indices = [26,27,34,35,42,43,52,53,61,62]
step!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [27,35,43,53,62]

number of geoms: 63
Simulation timestep: 0.01
Positions of joints: [-8.596852016136375e-6; -0.004081406032501581; -7.410299255294411e-6; -0.0017870869899753808; -0.0004647118334366869; -0.00011195763636581917; -7.5629513905124985e-6; -0.001822666571193126; -0.0004718530659545166; -0.00011346130615746156; 7.410299255294426e-6; -0.0017870869899753808; -0.0004647118334366869; -0.00011195763636581917; -0.002047871972935659; 7.0857582728000195e-6; -0.0012286533791773305; -0.0003163189196118475; -7.557667103994423e-5; 1.795065691926421e-5; -0.0014107106624610675; -0.0004629888303683335; 2.4753188382030703e-5; 5.000874638565504e-6;;]
Cartesion position of joints: [0.0 0.0 -0.1; 0.0061748421045633 -2.5023164499729428e-5 -0.0006274281832173423; 0.10662449729912864 8.494680512354149e-7 0.0008928917187008476; 0.0927456223223767 2.300031276486686e-8 2.164897719232844e-8; 0.181 0.0 0.01; 0.22907431384985827 -0.0026493678443454518 0.009977380725254718; 0.21301 0.0 0.01; 0.24701 0.026 0.01; 0.24701 -

5-element Vector{Int64}:
 27
 35
 43
 53
 62

In [6]:
data.geom_xpos[27,:]

3-element Vector{Float64}:
  0.42965647531732043
 -0.032976015650536074
  0.009579267426355864

In [7]:
target = [0.373;-0.404;0.066]

3-element Vector{Float64}:
  0.373
 -0.404
  0.066

## Test DIRCOL: moving index finger left right

In [90]:
function wrapped_mj_step(model, data, x, uk, forward)
    # given the current model and data. set the state and control to the model and perform a forward step
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        data.qpos .= x[1:model.nq]
        data.qvel .= x[(model.nq + 1):end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    if forward
        forward!(model,data)
    else
        step!(model, data) 
    end

    # return updated state k + 1
    finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return get_physics_state(model, data), finger_coordinates
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = params.data
    Q, R, Qf = params.Q, params.R, params.Qf

    # stage cost
    cost = 0.0
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        # # pass the full states through mujoco and get the finger coordinates
        _, finger_coordinates = wrapped_mj_step(model, data, xi, 0,true)

        # # get the index finger coordinates
        index_coordinates = finger_coordinates[1,:]

        # calculate the cost
        cost += 0.5*(index_coordinates-xg)'*Q*(index_coordinates-xg)+0.5*ui'*R*ui
    end

    # terminal cost 
    xt = Z[idx.x[N]]
    _, finger_coordinates = wrapped_mj_step(model, data, xt, 0, true)
    index_coordinates = finger_coordinates[1,:]
    cost += 0.5*(index_coordinates-xg)'*Qf*(index_coordinates-xg)
    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = params.data
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        # TODO: hermite simpson 
        # println("xi: ", xi[1:5])
        # print("ui: ", typeof(ui))
        xip1_mujoco, _ = wrapped_mj_step(model, data, xi, ui,false)
        c[idx.c[i]] = xip1_mujoco - xip1
    end

    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    data = params.data

    # _,initial_pose_index = wrapped_mj_step(model, data, Z[idx.x[1]], 0,true)
    # _,terminal_pose_index = wrapped_mj_step(model, data, Z[idx.x[N]], 0,true)
    
    # initial_index_coordinates = initial_pose_index[1,:]

    # terminal_index_coordinates = terminal_pose_index[1,:]

    # con_1 =  initial_index_coordinates - xic
    
    # con_2 =  terminal_index_coordinates - xg
    # con_1 = Z[idx.x[1]] - xic
    # con_2 = Z[idx.x[N]] - xg

    # return [con_1; con_2]
    return zeros(eltype(Z), 0)
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    # return inequality_constraints
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    data = params.data

    _,initial_pose_index = wrapped_mj_step(model, data, Z[idx.x[1]], 0,true)
    _,terminal_pose_index = wrapped_mj_step(model, data, Z[idx.x[N]], 0,true)
    
    initial_index_coordinates = initial_pose_index[1,:]

    terminal_index_coordinates = terminal_pose_index[1,:]

    con_1 =  initial_index_coordinates - xic
    
    con_2 =  terminal_index_coordinates - xg

    return [con_1; con_2; robohand_dynamics_constraints(params,Z)]
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [157]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("/home/aipex6/Conan/piano project/mujoco_menagerie/shadow_hand/scene_right_modified.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)
    
    # initiate time and time steps
    dt = 0.01
    tf = 0.1
    t_vec = 0:dt:tf
    N = length(t_vec)

    # Q size only 3 because we track one finger cartesian coordinates
    # Q = 0.5*diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))
    Q = 1*diagm(ones(3))
    R = 0.1*diagm(ones(model.nu))
    Qf = 10*diagm(ones(3))
    # indexing 
    idx = create_idx(model.nq + model.nv, model.nu, N)

    # calculate final state
    # start_data = data.geom_xpos[27,:]
    # end_data = [0.373;-0.404;0.066]
    step!(model, data)
    xic = get_physics_state(model, data)
    xic = data.geom_xpos[finger_geom_indices[1],:]
    U = zeros(20)
    U[9] = 0.664
    U[10] = 1.87
    U[12] = 0.664
    U[13] = 1.87
    # U[15] = 0.664
    # U[16] = 1.87
    # U[19] = 0.664
    # U[20] = 1.87
    for t in 1:9
        data.ctrl .= U
        step!(model, data)
    end
    # initial and goal state
    xg = get_physics_state(model, data)
    xg = data.geom_xpos[finger_geom_indices[1],:]
    # reset the model and data
    reset!(model, data)
    
    params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)


    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        for j = 1:model.nq
            x_l[idx.x[i][j]] = model.jnt_range[j,1]
            x_u[idx.x[i][j]] = model.jnt_range[j,2]
        end
    end
    for i = 1:(N-1)
        for j = 1:model.nu
            x_l[idx.u[i][j]] = model.actuator_ctrlrange[j,1]
            x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        end
    end
    # inequality constraints
    # c_l = -1 * Inf * ones(3*(idx.N-1))
    # c_u = Inf * ones(3*(idx.N-1))
    c_l = -0.1*ones(6+idx.nc)
    c_u = 0.1*ones(6+idx.nc)
    # c_l = zeros(0)
    # c_u = zeros(0)
    # initial guess
    step!(model, data)
    z0 = randn(idx.nz)
    # z0 .= get_physics_state(model, data)
    for i = 1:N
        z0[idx.x[i]] .= get_physics_state(model, data)
    end
    # println(length(robohand_inequality_constraints(params,z0)))
    for i = 1:N-1
        # for j = 1:model.nq
        #     z0[idx.x[i][j]] = rand(Uniform(model.jnt_range[j,1], model.jnt_range[j,2]))
        # end
        for j = 1:model.nu
            z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        end
    end
    # for j = 1:model.nq
    #     z0[idx.x[N][j]] = rand(Uniform(model.jnt_range[j,1], model.jnt_range[j,2]))
    # end
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    # diff_type = :auto
    diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-3, c_tol = 1e-2, max_iters = 100, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [158]:
idx = create_idx(model.nq + model.nv, model.nu, 10)
2+idx.nz

662

In [159]:
X, U, t_vec, params = solve_finger_moving(verbose=true) 

---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:   353808
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      728
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      464
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints.......

([[-0.13377771604658759, -0.212784504129804, -0.0037702303864757002, 0.5592356886026364, 0.7858259456481497, 0.8011710216397323, 0.0004307680285413085, 0.6538827103113498, 0.8141573331246034, 0.8206178949211327  …  2.6269532286309176, 2.786649050441702, -0.17328892782071856, 2.191310959278834, 1.5195249200469882, -0.3731825461296403, 0.982814406306067, 0.5564217597202386, -0.1394168955659979, 0.28154356114628865], [-0.13747174326049383, -0.16970756609103288, -0.0032175357970891036, 0.5799188474921939, 0.7667947356620375, 0.7770197303001523, 0.00041734865520119186, 0.6551162392590835, 0.7872230355706281, 0.790486239230005  …  -0.10435083642641402, 1.49778336879441, 0.29568569744240075, -1.623391578656625, -2.5043930103368703, -0.1380524350592599, 0.06564038457490017, 0.237051261349339, -0.014715984356608426, -0.7651555798446521], [-0.14349649945137194, -0.14151240323685527, -0.0023103964526271573, 0.5916280041037603, 0.7379247207282882, 0.7447609563349551, 0.0007031238599458285, 0.65007

In [151]:
U

10-element Vector{Vector{Float64}}:
 [0.0005172988473532852, -2.734347610054439e-5, -0.00030167403883407315, 0.004320020982756036, -2.740828617386874e-19, 1.7980865334768007e-6, -0.0001799648785453939, -2.1462644134603016e-14, -0.0001799637612016733, 0.004324569045236698, 1.704974470304354e-13, -0.00017996640794098236, 0.004324569045236698, -3.5099688187274824e-14, -0.0001799652626230555, 0.004324569045236698, 0.004315857536993242, 3.3376744361500973e-14, -0.00017996364491879371, 0.004324569045236698]
 [0.000517298847353229, -2.7286362007841953e-5, -0.0011317893415577016, 0.004320020982756036, -2.2206040082307154e-19, -2.968110952413465e-6, -0.0001799706753959178, -1.0222247225179264e-13, -0.00017997169702508795, 0.004324569045236698, 1.139754915305789e-13, -0.00017996663467032513, 0.004324569045236698, 5.502332437544764e-14, -0.0001799694255485082, 0.004324569045236698, 0.004315857536993242, -1.0362172901657721e-13, -0.0001799665273173488, 0.004324569045236698]
 [0.0005172988473532408

In [152]:
X

11-element Vector{Vector{Float64}}:
 [-0.17453300000000008, -0.10472000000000004, 1.9699556550749696e-17, 0.6545005028537489, 0.7854000028539971, 0.7854000028539971, -1.5573771468571745e-17, 0.6545005028537489, 0.7854000028539971, 0.7854000028539971  …  -0.20478719729356587, 0.0007085758272800019, -0.12286533791773305, -0.03163189196118475, -0.007557667103994423, 0.0017950656919264209, -0.14107106624610674, -0.04629888303683335, 0.00247531883820307, 0.0005000874638565504]
 [-0.1629378051120024, -0.09169673705595123, 0.0028161709306404846, 0.5751198976281718, 0.6841451859846237, 0.6792004425037388, 0.0029123507519839714, 0.5755234169176887, 0.6843722408617393, 0.6792599881355391  …  -3.925767077320439, -0.14065447543355997, -6.880075020330657, -10.023662882351514, -10.786262710609483, 0.4639392805968097, -7.276663002969711, 0.37850447799606, 0.35698540201771667, -9.024623936996166]
 [-0.14638437361455003, -0.077730602111421, 0.00459900980620864, 0.47698579036063776, 0.5649432522982659, 

In [164]:
# instantiate model and data
model = load_model("/home/aipex6/Conan/piano project/mujoco_menagerie/shadow_hand/scene_right_modified.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension
dt = 0.01
tf = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
reset!(model, data)
step!(model, data)
states[:,1] = get_physics_state(model, data)

# for t in 1:N-1
#     data.ctrl .= U[t][:]
#     step!(model, data)
#     states[:,t+1] = get_physics_state(model, data)
# end

for t in 1:N
    states[:,t] = X[t][:]
end

In [165]:
init_visualiser()
visualise!(model, data, trajectories = [states])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [30]:
# instantiate model and data
model = load_model("/home/aipex6/Conan/piano project/mujoco_menagerie/shadow_hand/scene_right_modified.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states = zeros(nx, 10)
states[:,1] = get_physics_state(model, data)
U = zeros(20)
U[9] = 0.664
U[10] = 1.87
# U[12] = 0.664
# U[13] = 1.87
# U[15] = 0.664
# U[16] = 1.87
# U[19] = 0.664
# U[20] = 1.87
for t in 1:9
    data.ctrl .= U
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end

In [31]:
init_visualiser()
visualise!(model, data, trajectories = [states])

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


In [195]:
function con(params, x)
    [
        robohand_equality_constraints(params, x);
        robohand_inequality_constraints(params, x)
    ]
end

model = load_model("piano_with_one_shadow_hand/scene.xml")
data = init_data(model)

# reset the model and data
reset!(model, data)

# initiate time and time steps
dt = 0.1
tf = 1.0
t_vec = 0:dt:tf
N = length(t_vec)

# LQR cost
# Q size only one because we track one finger
Q = diagm(ones(model.nq + model.nv))
R = 0.1*diagm(ones(model.nu))
Qf = 10*diagm(ones(model.nq + model.nv))

# indexing 
idx = create_idx(model.nq + model.nv, model.nu, N)
start_data = [data.qpos[:]; data.qvel[:]]
# calculate final state
start_pos = data.qpos
end_pos = start_pos
end_pos[1] = 0.2
end_pos[2] = 0.25
end_pos[8] = 1.0
end_pos[9] = 1.0
end_pos[10] = 1.0
end_pos[12] = 1.0
end_pos[13] = 1.0
end_pos[14] = 1.0
data.qpos[:] = end_pos[:]
step!(model, data)
end_data = [data.qpos[:]; data.qvel[:]]
reset!(model, data)
# initial and goal state
xic = start_data
xg = end_data

params = (Q = Q, R = R, Qf = Qf, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)
z0 = 0.001*randn(idx.nz)
println(size(z0))
# ForwardDiff.gradient(_x -> robohand_cost(params, _x), z0)
ForwardDiff.jacobian(_x -> robohand_dynamics_constraints(params, _x), z0)

(2728,)


2280×2728 Matrix{Float64}:
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  ⋮                             ⋮    ⋱                     ⋮          
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0     -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
 -